In [1]:
import os
import sys
# # sys.setrecursionlimit(10000)
from time import time
import numpy as np
from Tree.tree_builder import tree_builder
from Tree.tree_values import TreeValues
# from Tree.strategy_filling import strategy_filling
from Tree.tree_visualizer import tree_visualizer
from Tree.tree_cfr import TreeCFR
from helper_classes import TreeParams, Node
from Game.card_to_string_conversion import card_to_string
from Game.card_tools import card_tools
from Settings.constants import constants
from Settings.arguments import arguments
from Settings.game_settings import game_settings
# from Nn.value_nn import ValueNn
# from Nn.next_round_value import NextRoundValue
# from Nn.mock_nn_terminal import MockNnTerminal
# from TerminalEquity.terminal_equity import TerminalEquity
# from Training.train import Train
# from Tree.tree_strategy_filling import TreeStrategyFilling
# from Game.Evaluation.evaluator import evaluator

# Random functions

In [2]:
# card_to_string.string_to_board('KsKhJs')

In [3]:
# card_tools.string_to_hole_index('Jh')

In [4]:
# a = card_tools.get_next_round_boards(np.zeros([]))
# a[-5:]

In [5]:
# card_tools.get_last_boards_count(1)

In [6]:
# card_tools.get_flop_board_index(np.array([10,20,30])-1)

In [7]:
# card_tools.get_hole_index([11,14])

In [8]:
# card_tools.get_board_index(np.array([2,20,30,40,10])-1)

In [9]:
# evaluator.evaluate_seven_card_hand(np.array([2,4,8,16,32,15,7])-1)

In [10]:
# a = np.arange(6*7).reshape([6,7])
# evaluator.evaluate_fast(a)

In [11]:
# evaluator.batch_eval_fast(np.array([0,1,2]))[783]

In [12]:
PC = constants.players_count
CC = game_settings.card_count
HC = game_settings.hand_count
P1 = constants.players.P1
P2 = constants.players.P2

# Tree Build

In [13]:
params = TreeParams()
first_node = Node()
first_node.board = np.zeros([], dtype=int)
first_node.street = 1
first_node.current_player = constants.players.P1
first_node.bets = np.array([50, 100], dtype=int)
first_node.num_bets = 0
params.root_node = first_node
params.limit_to_street = True

root = tree_builder.build_tree(params)

In [33]:
# tree_visualizer.draw_tree(root, size='30,10', show_vars=True)

# CFR

In [15]:
card_to_string.string_to_board('6c6d2c2d3h')

array([16, 17,  0,  1,  6], dtype=int16)

In [16]:
params = TreeParams()

params.root_node = Node()
params.root_node.board = card_to_string.string_to_board('6c6d2c2d3h')
params.root_node.street = 4
params.root_node.current_player = constants.players.P1
params.root_node.bets = np.array([4, 4])
params.root_node.num_bets = 0

tree = tree_builder.build_tree(params)

starting_ranges = np.zeros([PC,HC], dtype=arguments.dtype)
starting_ranges[0] = card_tools.get_uniform_range(params.root_node.board)
starting_ranges[1] = card_tools.get_uniform_range(params.root_node.board)

t0 = time()
tree_cfr = TreeCFR()
# %lprun -f tree_cfr.cfrs_iter_dfs tree_cfr.run_cfr(tree, starting_ranges)
tree_cfr.run_cfr(tree, starting_ranges)
print('Took time: {} for {} iters'.format(time()-t0, arguments.cfr_iters))

tree_values = TreeValues()
tree_values.compute_values(tree, starting_ranges)

print('Exploitability: ' + str(tree.exploitability) + ' [chips]' )

Took time: 6.368797540664673 for 1000 iters
Exploitability: 0.887353 [chips]


In [17]:
print(np.array2string(tree.strategy[:, 522:527], suppress_small=True, precision=5))

[[0.42921 0.42921 0.42921 0.42921 0.42921]
 [0.57079 0.57079 0.57079 0.57079 0.57079]
 [0.      0.      0.      0.      0.     ]]


# Card and Range Generator

In [18]:
# from DataGeneration.random_card_generator import card_generator
# card_generator.generate_cards(6)

In [19]:
# from DataGeneration.range_generator import RangeGenerator
# r = RangeGenerator()
# board = np.zeros([])
# r.set_board(board)
# BS,CC = 10,6
# ranges = np.zeros([BS,CC])
# r.generate_range(ranges)
# print(np.array2string(ranges, suppress_small=True, precision=2))
# print(np.array2string(np.sum(ranges, axis=1, keepdims=True), suppress_small=True, precision=2))

# Lookahead (test 1)

In [20]:
# params = TreeParams()
# params.root_node = Node()
# params.root_node.board = card_to_string.string_to_board('Ks')
# params.root_node.street = 2
# params.root_node.current_player = constants.players.P1
# params.root_node.bets = np.array([100, 100])
# params.limit_to_street = False
# tree = tree_builder.build_tree(params)

In [21]:
# from Tree.tree_strategy_filling import TreeStrategyFilling

# filling = TreeStrategyFilling()

# range1 = card_tools.get_uniform_range(params.root_node.board)
# range2 = card_tools.get_uniform_range(params.root_node.board)

# filling.fill_strategies(tree, P1, range1, range2)
# filling.fill_strategies(tree, P2, range1, range2)

In [22]:
# starting_ranges = np.zeros([PC,CC], dtype=arguments.dtype)
# starting_ranges[0] = card_tools.get_uniform_range(params.root_node.board)
# starting_ranges[1] = card_tools.get_uniform_range(params.root_node.board)

# tree_values = TreeValues()
# tree_values.compute_values(tree, starting_ranges)

# print('Exploitability: ' + str(tree.exploitability) + ' [chips]' )

# Next Round Value Bucketing test

In [23]:
# next_round_value = NextRoundValue(None)
# print(next_round_value._range_matrix)
# # test of card to bucket range translation
# range_ = np.arange(1,7).reshape([1,-1]).astype(float)
# next_round_range = np.zeros([1, next_round_value.bucket_count * next_round_value.board_count], dtype=arguments.dtype)
# next_round_value._card_range_to_bucket_range(range_, next_round_range)
# print(next_round_range)

# Next Round Value (mock test)

In [24]:
# # # test of get_value functionality
# mock_nn = MockNnTerminal()
# # mock_nn = ValueNn()
# next_round_value = NextRoundValue(mock_nn)

# # bets = np.array([100], dtype=float)
# bets = np.array([1200], dtype=float)

# next_round_value.start_computation(bets)

# ranges = np.full([1,2,CC], 1/4, dtype=arguments.dtype)
# values = np.zeros([1,2,CC], dtype=arguments.dtype)

# np.random.seed(0)
# ranges[0][0] = np.array([1,1,0,0,0,0])
# ranges[0][1] = np.array([1,1,1,1,1,1])

# next_round_value.get_value(ranges, values)

# print(values)

# ranges_2 = ranges.reshape([2,CC]).copy()
# values_2 = np.full_like(ranges_2, -1)

# terminal_equity = TerminalEquity()
# terminal_equity.set_board(np.zeros([]))
# terminal_equity.call_value(ranges_2, values_2)
# print('terminal_equity')
# print(values_2)

# # output:
# # [[[ 2.  2.  0.  0. -2. -2.]
# #   [ 0.  0. -1. -1. -1. -1.]]]
# # terminal_equity
# # [[ 0.  0. -1. -1. -1. -1.]
# #  [ 2.  2.  0.  0. -2. -2.]]

In [25]:
# # needs to comment out some asserts and use_memory = True

# board = card_to_string.string_to_board('Ks')

# values_3 = np.full_like(values, -1)
# next_round_value.get_value_on_board(board, values_3)

# print(values_3)

# # output:
# # [[[ 0.2  0.2  0.   0.8 -0.6 -0.6]
# #   [ 0.   0.   0.   1.  -1.  -1. ]]]

# Nn output

In [26]:
# data_dir = os.path.join(arguments.data_path, 'tfrecords')
# T = Train(data_dir=data_dir)
# T.train(num_epochs=100, verbose=0)
# model = T.keras_model

In [27]:
# npy_folder = r'C:\Users\domin\Documents\GitHub\PyStack\Data\TrainSamples\PotBet\npy'
# m = np.load(npy_folder + r'\masks.0.npy')
# x = np.load(npy_folder + r'\inputs.0.npy')
# y = np.load(npy_folder + r'\targets.0.npy')

# y_hat = model.predict(x)

# temp = m[0]
# m = np.zeros([72])
# m[:36] = temp
# m[36:] = temp
# x = x[0]
# y = y[0]
# y_hat = y_hat[0]

# d = np.dot(y_hat, x[:-1])
# d /= 2
# y_new = y_hat - d
# y_new = y_hat

# (y_new*m)[:36] - (y_new*m)[36:]

# Continual Re-solving in Tree

In [28]:
# board = card_to_string.string_to_board('')

In [29]:
# params = TreeParams()
# params.root_node = Node()
# params.root_node.board = board
# params.root_node.street = 1
# params.root_node.current_player = constants.players.P1
# params.root_node.bets = np.array([300, 300])
# params.limit_to_street = False
# tree = tree_builder.build_tree(params)

In [30]:
# filling = TreeStrategyFilling()

# range1 = card_tools.get_uniform_range(board)
# range2 = card_tools.get_uniform_range(board)

# filling.fill_strategies(tree, P1, range1, range2)
# filling.fill_strategies(tree, P2, range1, range2)

In [31]:
# starting_ranges = np.zeros([PC,CC], dtype=arguments.dtype)
# starting_ranges[0] = card_tools.get_uniform_range(params.root_node.board)
# starting_ranges[1] = card_tools.get_uniform_range(params.root_node.board)

# tree_values = TreeValues()
# tree_values.compute_values(tree, starting_ranges)

# print('Exploitability: ' + str(tree.exploitability) + ' [chips]' )

In [32]:
# tree_visualizer.draw_tree(tree, size='20,10')